# Classification of CIFAR10 Dataset using Convolutional Neural Networks

## Introduction
The CIFAR10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

We will use a Convolutional Neural Network (CNN) to classify the images in the CIFAR10 dataset.

## 1. Set Up
Install and import the necessary libraries.

In [11]:
# Install the required libraries

#!pip install torch
#!pip install torchvision

In [12]:
import torch
import torchvision

from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, Subset

In [13]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.beckends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2. Import Data
We'll download the CIFAR10 dataset from teh torchvision library.

**Note: CIFAR10 dataset input size is 3x32x32**

In [14]:
# Set image path if using google drive
img_path = '/content/data'

In [15]:
# Set image path if using local device
#img_path = './data'

In [16]:
# Prepare to transform the image to tensors
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        ),
    ]
)

# Download CIFAR10 train dataset
train_dataset = torchvision.datasets.CIFAR10(root=img_path, train=True, download=True, transform=transform)
train_dataset = Subset(train_dataset, range(10000)) # Use 10,000 images instead of the total 50,0000

# Download CIFAR10 test dataset
test_dataset = torchvision.datasets.CIFAR10(root=img_path, train=False, download=True, transform=transform)
test_dataset = Subset(test_dataset, range(1000)) # Use 1,000 images instead of the total 10,000

# Load training dataset
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=512,
    shuffle=True
)

# Load testing dataset
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=512,
    shuffle=False
)

Files already downloaded and verified
Files already downloaded and verified


## 3. Convolutional Neural Network (CNN)

We will define a CNN with the following architecture<br>
* Convolutional Layer 1
* Convolutional Layer 2
* Max Pooling 1
* Convolutional Layer 3
* Convolutional Layer 4
* Max Pooling 2
* Flatten
* Fully Connected Layer 1
* Fully Connected Layer 2
* Fully Connected Layer 3




### 3.1 Create CNN Class

In [17]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.Sequential = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1, stride=1), # Input: 3x32x32, Output: 32x32x32
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, stride=1), # Input: 32x32x32, Output: 32x32x32
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # Input: 32x32x32, Output: 32x16x16
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1), # Input: 32x16x16, Output: 64x16x16
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1), # Input: 64x16x16, Output: 64x16x16
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # Input: 64x16x16, Output: 64x8x8
        nn.Flatten(), # Output: 64*8*8
        nn.Linear(in_features=64*8*8, out_features=1024), # Input: 63*8*8, Output: 1024
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=1024, out_features=512), # Input: 1028, Output: 512
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=10) # Input: 128, Output: 10
    )

  def forward(self, x):
    return self.Sequential(x)

## 3.2 Train the Model

In [19]:
# Initialize model
model = ConvNet()

# Set optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Set loss function
loss_fn = nn.CrossEntropyLoss()

# Set number of epochs
epochs = 20

# Train model
for epoch in range(epochs):
  # Track accuracy
  accuracy_history = 0

  for batch_idx, (data, targets) in enumerate(train_loader):
    # Forward pass
    pred = model(data)

    # Calculate loss
    loss = loss_fn(pred, targets)

    # Run backprogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Calculate accuracy
    _, predicted = torch.max(pred.data, 1)
    accuracy = (predicted == targets).sum().item() / targets.size(0)
    accuracy_history += accuracy

  print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {accuracy_history/len(train_loader):.4f}")

Epoch: 1, Loss: 2.2832, Accuracy: 0.1261
Epoch: 2, Loss: 2.1517, Accuracy: 0.1830
Epoch: 3, Loss: 1.9857, Accuracy: 0.2447
Epoch: 4, Loss: 1.8417, Accuracy: 0.3060
Epoch: 5, Loss: 1.8888, Accuracy: 0.3248
Epoch: 6, Loss: 1.6405, Accuracy: 0.3699
Epoch: 7, Loss: 1.4457, Accuracy: 0.4405
Epoch: 8, Loss: 1.3591, Accuracy: 0.4790
Epoch: 9, Loss: 1.4932, Accuracy: 0.5183
Epoch: 10, Loss: 1.1058, Accuracy: 0.5542
Epoch: 11, Loss: 1.1055, Accuracy: 0.6132
Epoch: 12, Loss: 1.0621, Accuracy: 0.6486
Epoch: 13, Loss: 0.8866, Accuracy: 0.7088
Epoch: 14, Loss: 0.7068, Accuracy: 0.7547
Epoch: 15, Loss: 0.7408, Accuracy: 0.7901
Epoch: 16, Loss: 0.4709, Accuracy: 0.8187
Epoch: 17, Loss: 0.4980, Accuracy: 0.8491
Epoch: 18, Loss: 0.2772, Accuracy: 0.8884
Epoch: 19, Loss: 0.4417, Accuracy: 0.8984
Epoch: 20, Loss: 0.3357, Accuracy: 0.9002
